# Pre-trained Model Exploration

- Explore some generative models in action

## StyleGAN - Faces

In [ ]:
!git clone https://github.com/NVlabs/stylegan.git
# conda activate aihub_tf1

In [1]:
import os
import pickle
import warnings
import numpy as np
import PIL

In [2]:
from tensorflow.python.util import module_wrapper
module_wrapper._PER_MODULE_WARNING_LIMIT = 0

In [3]:
import stylegan
from stylegan.dnnlib import tflib
from stylegan import config

tflib.init_tf()

# Move into the StyleGAN directory, if you're not in it already
path = 'stylegan/'
if "stylegan" not in os.getcwd():
    os.chdir(path)

In [4]:
# Load pre-trained StyleGAN network
url = 'https://bitbucket.org/ezelikman/gans/downloads/karras2019stylegan-ffhq-1024x1024.pkl' # karras2019stylegan-ffhq-1024x1024.pkl
with stylegan.dnnlib.util.open_url(url, cache_dir=stylegan.config.cache_dir) as f:
    # You'll load 3 components, and use the last one Gs for sampling images.
    #   _G = Instantaneous snapshot of the generator. Mainly useful for resuming a previous training run.
    #   _D = Instantaneous snapshot of the discriminator. Mainly useful for resuming a previous training run.
    #   Gs = Long-term average of the generator. Yields higher-quality results than the instantaneous snapshot.
    _G, _D, Gs = pickle.load(f)

    print('StyleGAN package loaded successfully!')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
StyleGAN package loaded successfully!


In [5]:
#@title Generate faces with StyleGAN
#@markdown Double click here to see the code. After setting truncation, run the cells below to generate images. This adjusts the truncation, you will learn more about this soon! Truncation trades off fidelity (quality) and diversity of the generated images - play with it!

Truncation = 0.7 #@param {type:"slider", min:0.1, max:1, step:0.1}

print(f'Truncation set to {Truncation}. \nNow run the cells below to generate images with this truncation value.')

Truncation set to 0.7. 
Now run the cells below to generate images with this truncation value.


In [6]:
# Set the random state. Nothing special about 42,
#   except that it's the meaning of life.
rnd = np.random.RandomState(42)

print(f'Random state is set.')

Random state is set.


In [7]:
batch_size = 4 #@param {type:"slider", min:1, max:10, step:1}

print(f'Batch size is {batch_size}...')

Batch size is 4...


- Noise vectors make sure the generated images are randomly - stochastically different, not all the same
- There is a noise vector for each image in the batch
- Run the next cell to get new noise vectors and asa result new images

In [10]:
input_shape = Gs.input_shape[1]
noise_vectors = rnd.randn(batch_size, input_shape)

print(f'There are {noise_vectors.shape[0]} noise vectors, each with {noise_vectors.shape[1]} random values between -{Truncation} and {Truncation}.')

There are 4 noise vectors, each with 512 random values between -0.7 and 0.7.


- Run the model to generate the images
- Notice that truncation and noise vectors are passed in
- Focus on output formats and adding additional randomness/diversity to the output

In [12]:
fmt = dict(func=tflib.convert_images_from_uint8, nchw_to_nhwc=True)
images = Gs.run(noise_vectors, None, truncation_psi=Truncation, randomize_noise=False, ouput_tranform=fmt)

print(f'Successfully sampled {batch_size} images from the model.')

Successfully sampled 4 images from the model.


In [ ]:
os.makedires(config.result_dir, exist_ok=True)
png_filename = os.path.join(config.result_dir, 'stylegan-example.png')

if(batch_size > 1):
    img = np.concatenate(images, axis=1)
else:
    img = images[0]
    
PIL.Image.fromarray(img, 'RGB').save(png_filename)

# Check the images out
from IPython.display import Image
Image(png_filename, width=256*batch_size, height=256)

ModuleNotFoundError: No module named 'tensorflow_hub'